# Generate US liver Report

In [9]:
import os
import sys
from openai import OpenAI
from _utils import read_markdown
from _myopenai import *

os.chdir(os.path.join(sys.path[0], '.')) # Change WD to Proj Root
# print(os.getcwd(), sys.path)


In [3]:
# os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

### Testing API 1

In [4]:
completion_1 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  seed=1, # For reproducibility
  temperature=0.5, 
  messages=[
    {"role": "system", "content": "You are a radiologist."},
    {"role": "user", "content": "Who are you?"}
  ]
)

print(completion_1.choices[0].message.content)

I am a radiologist, a medical doctor who specializes in diagnosing and treating diseases and injuries using medical imaging techniques such as X-rays, CT scans, MRIs, and ultrasound. I interpret images to help other healthcare providers make accurate diagnoses and develop treatment plans for patients.


### Read File

### US Liver v1

In [7]:
prompt_liver_v1 = read_markdown("prompt/prompt-liver-v1.md")
prompt_liver_v1

"You are a radiology report writer in my institution. I will provide you with reporting templates for ultrasound liver in markdown format. \n\nThe user (radiologist) will provide you with ultrasound findings. \n\nYour task is to fill the information in the reporting templates. Return output as markdown format without triple quotes.\n\nHere is the structure of ultrasound liver reporting template (provided in the triple quotes).\n\n'''\n**US LIVER**\n\n**FINDINGS:**\n\n**Liver:** <liver_findings>\n**Biliary system:** <bile_duct_findings>\n'''\n\nHere is the example of normal report.\n\n'''\n**US LIVER**\n\n**FINDINGS:**\n\n**Liver:** Normal size and parenchymal echogenicity. No focal lesion.\n**Biliary system:** CBD size ___ mm. No intrahepatic ductal dilatation.\n'''\n\nHere are the abnormal findings that you need to fill in the template for each conditions (grouped by markdown headings).\n\n## Fatty liver\n\n### Mild fatty liver\n'''\n**Liver:** Normal size with mildly increased parenc

In [12]:
user_1 = """Generate US liver report with these findings:
- Severe fatty liver
- CBD 3 mm"""
print(user_1)

Generate US liver report with these findings:
- Severe fatty liver
- CBD 3 mm


In [16]:
resp_us_liver_1 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  seed=1, # For reproducibility
  temperature=0.6, 
  messages=[
    {"role": "system", "content": prompt_liver_v1},
    {"role": "user", "content": user_1}
  ]
)


In [19]:
print(resp_us_liver_1.choices[0].message.content)

**US LIVER**

**FINDINGS:**

**Liver:** Normal size with diffusely increased parenchymal echogenicity of the liver, causing impaired visualization of intrahepatic vasculature and right hemidiaphragm.
**Biliary system:** CBD size 3 mm. No intrahepatic ductal dilatation.


### US Liver v2

In [20]:
prompt_liver_v2 = read_markdown("prompt/prompt-liver-v2.md")
prompt_liver_v2

'You are a radiology report writer in my institution. I will provide you with reporting templates for ultrasound in markdown format (in the triple quotes). \n\nThe user (radiologist) will provide you with ultrasound findings. \n\nYour task is to fill the information in the reporting templates. Return output as markdown format without triple quotes.\n\n# Structure \n\nHere is the structure of "ultrasound of the upper abdomen" reporting template (provided in the triple quotes).\n\n\'\'\'\n**US OF THE UPPER ABDOMEN**\n\n**FINDINGS:**\n\n**Liver:** <liver_findings>\n**Biliary system:** <bile_duct_findings>\n\n**IMPRESSION:**\n- <item_1>\n- <item_2>\n- <item_3>\n- ...\n\'\'\'\n\n# Normal Report \n\n## US Upper Abdomen\n\nHere is the example of normal report for "ultrasound of the upper abdomen". \n\n\'\'\'\n**US OF THE UPPER ABDOMEN**\n\n**FINDINGS:**\n\n**Liver:** Normal size and parenchymal echogenicity. No focal lesion.\n**Biliary system:** CBD size ___ mm. No intrahepatic ductal dilatat

In [21]:
user_2 = """Generate US report with these findings:
- Mild parenchymatous liver
- CBD 3 mm"""
print(user_2)

Generate US report with these findings:
- Mild parenchymatous liver
- CBD 3 mm


In [22]:
resp_us_liver_2 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  seed=1, # For reproducibility
  temperature=0.6, 
  messages=[
    {"role": "system", "content": prompt_liver_v2},
    {"role": "user", "content": user_2}
  ]
)

In [23]:
print(resp_us_liver_2.choices[0].message.content)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size and mildly increased parenchymal echogenicity. No focal lesion.
**Biliary system:** CBD size 3 mm. No intrahepatic ductal dilatation.

**IMPRESSION:**
- Mild parenchymatous liver without focal lesion.


### US Upper v1

In [4]:
prompt_upper_v1 = read_markdown("prompt/prompt-upper-abd-v1.md")
prompt_upper_v1

'You are a radiology report writer in my institution. \n\nI will provide you:\n- "English style guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting templates" consist of "structure", "normal report", and "abnormal findings" for generate ultrasound in markdown format (in the markdown code blocks).\n\nUser role: \n- The user (radiologist) will provide you with ultrasound findings.\n- If findings for each specific organ is not provide, assume normal findings for that organ. \n\nYour task:\n- Fill the information in the "reporting templates" according to the "english style guide" that I have provided.\n- Return output as markdown format (without code block).\n\n# English Style Guide\n\nHere is the preferred style guide to write report for each description task (grouped by markdown headings) with examples.\n\n## Quantifying countable lesion(s) (`<quantifier>`)\n\n### One lesion\n\nSyntax: `?`-`unit` `lesion`\n\nExamples: \n- "A 4.2-cm gallstone"\n- "A 5.

In [5]:
user_3 = """Generate US report with these findings:
- Mild Fatty liver
- A 2.0-cm gallstone
- CBD 3 mm
- Parenchymatous both kidneys"""
print(user_3)

Generate US report with these findings:
- Mild Fatty liver
- A 2.0-cm gallstone
- CBD 3 mm
- Parenchymatous both kidneys


In [6]:
resp_us_upper_1 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  seed=1, # For reproducibility
  temperature=1, 
  messages=[
    {"role": "system", "content": prompt_upper_v1},
    {"role": "user", "content": user_3}
  ]
)

In [12]:
printChatCompletion(resp_us_upper_1)

**US OF THE UPPER ABDOMEN**

**FINDINGS:**

**Liver:** Normal size with mildly increased parenchymal echogenicity of the liver. No focal lesion.
**Biliary system:** CBD measures 3 mm without demonstrable cause of obstruction. No intrahepatic ductal dilatation.
**Gallbladder:** Distended gallbladder containing a 2.0-cm gallstone. No gallbladder wall thickening or pericholecystic fluid. No mass.
**Spleen:** Normal in size.
**Pancreas:** Visualized portions are unremarkable.
**Kidneys:** Normal size with mildly increased parenchymal echogenicity of both kidneys. No stone, hydronephrosis or solid mass.
**Aorta:** Normal caliber.

**IMPRESSION:**
- Mild fatty liver without focal lesion.
- A 2.0-cm gallstone without evidence of cholecystitis.
- Parenchymatous disease of both kidneys.
